## Installing libraries

In [55]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import threading
import math

from scipy.stats import kendalltau
from lime import lime_image
from tensorflow.keras import optimizers
from scipy.stats import kendalltau
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array

from FunctionReference import CBAM_Model, SE_Model, Se_functional_model, Cbam_functional_model

## Meta parameters

In [72]:
model_dir = "..\\Models\\TrainedModels\\"
dataset_dir = "Subset"

saving_treshold = 10

## Loading models

In [54]:
cbam_model = load_model(model_dir + "TrainedCbamModel.keras", compile=True)
se_model = load_model(model_dir + "TrainedSeModel.keras", compile=True)

ValueError: File not found: filepath=..\Models\TrainedModels\TrainedCbamModel.keras. Please ensure the file is an accessible `.keras` zip file.

## Creating functional model and copying weights

In [ ]:
functional_cbam_model = cbam_functional_model(LABEL_CLASS, IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)
functional_se_model = Se_functional_model(LABEL_CLASS, IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)

In [ ]:
for layer in functional_cbam_model.layers:
    try:
        layer.set_weights(cbam_model.get_layer(layer.name).get_weights())
    except (ValueError, AttributeError):
        print(layer.name)
        pass

In [ ]:
for layer in functional_se_model.layers:
    try:
        layer.set_weights(se_model.get_layer(layer.name).get_weights())
    except (ValueError, AttributeError):
        print(layer.name)
        pass

## Loading dataset

In [59]:
def get_directories_images(path):
    images = []

    for Image in os.listdir(path + "\\"):
        images.append(path + "\\" + Image)

    return pd.DataFrame({'images': images})

In [60]:
df = get_directories_images(dataset_dir)

## Explanation techniques

### Grad-CAM

In [61]:
def get_gradcam(model, image, layer_name):      
    # Grad-CAM
    explainer = GradCAM()
    predictions = model.predict(image)
    predicted_class = np.argmax(predictions[0])

    # Apply Grad-CAM to obtain the heatmap
    cams = explainer.explain(
        validation_data=(image, None),
        model=model,
        layer_name=layer_name,
        class_index=predicted_class
    )

    # Post processing of the heatmap
    heatmap = cams[0].numpy()
    height, width = image.shape[:2]

    if heatmap.shape != (height, width):
        heatmap = cv2.resize(heatmap, (width, height))

    return heatmap

### LIME

In [62]:
def get_lime(model, image_array_u, image_input_u):
    explainer = lime_image.LimeImageExplainer()

    # Explain the instance
    explanation = explainer.explain_instance(
        image_array_u,         
        model.predict,
        top_labels=26,
        hide_color=0,
        num_samples=1000
    )

    # Get explanation for the top predicted class
    label = explanation.top_labels[0]
    dict_heatmap = dict(explanation.local_exp[label])
    segments = explanation.segments

    # Create pixel-wise heatmap from superpixel weights
    heatmap = np.zeros(segments.shape)
    for segment_id, weight in dict_heatmap.items():
        heatmap[segments == segment_id] = weight

    only_positive_heatmap = np.maximum(heatmap, 0)
    
    return only_positive_heatmap 

### Integrated Gradients

In [63]:
def get_integrated_gradients(model, image, target_class_index, steps=50):    
    baseline = np.zeros_like(image)  # All zeros baseline
    image_input = tf.convert_to_tensor(image, dtype=tf.float32)  # Convert input image to tensor
    
      # Interpolate between baseline and image
    interpolated_images = [(baseline + (step / steps) * (image - baseline)) for step in range(steps)]
    interpolated_images = np.array(interpolated_images)

    # Convert the interpolated images to TensorFlow tensors
    interpolated_images = tf.convert_to_tensor(interpolated_images, dtype=tf.float32)

    # Remove extra dimension to match the input shape expected by the model
    interpolated_images = tf.squeeze(interpolated_images, axis=1)  # This removes the extra 1 dimension

    # Compute gradients
    with tf.GradientTape() as tape:
        tape.watch(interpolated_images)
        predictions = model(interpolated_images)
        target_class_probabilities = predictions[:, target_class_index]

    grads = tape.gradient(target_class_probabilities, interpolated_images)
    integrated_grads = np.mean(grads, axis=0) * (image - baseline)

    heatmap = np.sum(integrated_grads, axis=-1)
    integrated_gradients_heatmap = np.squeeze(heatmap)

    positive_only_heatmap = np.maximum(integrated_gradients_heatmap, 0)

    return positive_only_heatmap

## Utility functions

In [64]:
def scale_up(array, target_size=64):
    y = target_size // array.shape[0]
    x = target_size // array.shape[1]
    
    return np.kron(array, np.ones((y, x)))

In [65]:
def condense(spatial_attention, technique='avg'):
    weight = -1
    
    if technique == 'avg':
        weight = np.mean(spatial_attention)
    elif technique == 'max':
        weight = np.max(spatial_attention)
    elif technique == 'min':
        weight = np.min(spatial_attention)
    
    return weight

## Calculating similarities using multi-threading

### Wrapper for running functions concurrently

In [66]:
def save_attention(model, image):
    _ = model(image, save_attention=True)

In [67]:
def run_with_return(target_func, return_dict, key, *args):
    result = target_func(*args)
    return_dict[key] = result

### Performing calculations, continuosly saving the data

In [74]:
images_processed = 0
images_treshold = math.ceil(len(df) / saving_treshold)

explanation_names = ['gradcam', 'lime', 'ig']

model_names = []
attention_types = []
layer_nums = []
explanation_types = []
scores = []

model_names1 = []
gradcam0_lime_scores = []
gradcam1_lime_scores = []
gradcam2_lime_scores = []
gradcam3_lime_scores = []
gradcam4_lime_scores = []
gradcam0_ig_scores = []
gradcam1_ig_scores = []
gradcam2_ig_scores = []
gradcam3_ig_scores = []
gradcam4_ig_scores = []
lime_ig_scores = []

for index, image_dir in df.iterrows():
    image = load_img(image_dir.iloc[0], target_size=(64, 64))
    image_array = img_to_array(image) / 255.0
    image_input = np.expand_dims(image, axis=0)

    cbam_explanations = {}
    se_explanations = {}

    cbam_threads = [
        threading.Thread(target=run_with_return, args=(get_gradcam, cbam_explanations, "gradcam0", functional_cbam_model, image_input, 0)),
        threading.Thread(target=run_with_return, args=(get_gradcam, cbam_explanations, "gradcam1", functional_cbam_model, image_input, 1)),
        threading.Thread(target=run_with_return, args=(get_gradcam, cbam_explanations, "gradcam2", functional_cbam_model, image_input, 2)),
        threading.Thread(target=run_with_return, args=(get_gradcam, cbam_explanations, "gradcam3", functional_cbam_model, image_input, 3)),
        threading.Thread(target=run_with_return, args=(get_gradcam, cbam_explanations, "gradcam4", functional_cbam_model, image_input, 4)),
        threading.Thread(target=run_with_return, args=(get_lime, cbam_explanations, "lime", cbam_model, image_array, image_input)),
        threading.Thread(target=run_with_return, args=(get_integrated_gradients, cbam_explanations, "ig", cbam_model, image_input, target_class_index)),
        threading.Thread(target=save_attention, args=(cbam_model, image_input))
    ]

    se_threads = [
        threading.Thread(target=run_with_return, args=(get_gradcam, se_explanations, "gradcam0", functional_se_model, image_input, 0)),
        threading.Thread(target=run_with_return, args=(get_gradcam, se_explanations, "gradcam1", functional_se_model, image_input, 1)),
        threading.Thread(target=run_with_return, args=(get_gradcam, se_explanations, "gradcam2", functional_se_model, image_input, 2)),
        threading.Thread(target=run_with_return, args=(get_gradcam, se_explanations, "gradcam3", functional_se_model, image_input, 3)),
        threading.Thread(target=run_with_return, args=(get_gradcam, se_explanations, "gradcam4", functional_se_model, image_input, 4)),
        threading.Thread(target=run_with_return, args=(get_lime, se_explanations, "lime", se_model, image_array, image_input)),
        threading.Thread(target=run_with_return, args=(get_integrated_gradients, se_explanations, "ig", se_model, image_input, target_class_index)),
        threading.Thread(target=save_attention, args=(se_model, image_input))
    ]

    for thread in cbam_threads + se_threads:
        t.start()

    for thread in cbam_threads + se_threads:
        t.join()

    # Calculating the similarities between different explanation techniques
    se_gradcam0_lime_score, _ = kendalltau(se_explanations['gradcam0'].flatten(), se_explanations['lime'].flatten())
    se_gradcam1_lime_score, _ = kendalltau(se_explanations['gradcam1'].flatten(), se_explanations['lime'].flatten())
    se_gradcam2_lime_score, _ = kendalltau(se_explanations['gradcam2'].flatten(), se_explanations['lime'].flatten())
    se_gradcam3_lime_score, _ = kendalltau(se_explanations['gradcam3'].flatten(), se_explanations['lime'].flatten())
    se_gradcam4_lime_score, _ = kendalltau(se_explanations['gradcam4'].flatten(), se_explanations['lime'].flatten())
    se_gradcam0_ig_score, _ = kendalltau(se_explanations['gradcam0'].flatten(), se_explanations['ig'].flatten())
    se_gradcam1_ig_score, _ = kendalltau(se_explanations['gradcam1'].flatten(), se_explanations['ig'].flatten())
    se_gradcam2_ig_score, _ = kendalltau(se_explanations['gradcam2'].flatten(), se_explanations['ig'].flatten())
    se_gradcam3_ig_score, _ = kendalltau(se_explanations['gradcam3'].flatten(), se_explanations['ig'].flatten())
    se_gradcam4_ig_score, _ = kendalltau(se_explanations['gradcam4'].flatten(), se_explanations['ig'].flatten())
    se_lime_ig_score, _ = kendalltau(se_explanations['lime'].flatten(), se_explanations['ig'].flatten())

    
    model_names1.append('SE')
    gradcam0_lime_scores.append(se_gradcam0_lime_score)
    gradcam1_lime_scores.append(se_gradcam1_lime_score)
    gradcam2_lime_scores.append(se_gradcam2_lime_score)
    gradcam3_lime_scores.append(se_gradcam3_lime_score)
    gradcam4_lime_scores.append(se_gradcam4_lime_score)
    gradcam0_ig_scores.append(se_gradcam0_ig_score)
    gradcam1_ig_scores.append(se_gradcam1_ig_score)
    gradcam2_ig_scores.append(se_gradcam2_ig_score)
    gradcam3_ig_scores.append(se_gradcam3_ig_score)
    gradcam4_ig_scores.append(se_gradcam4_ig_score)
    lime_ig_scores.append(se_lime_ig_score)

    cbam_gradcam0_lime_score, _ = kendalltau(se_explanations['gradcam0'].flatten(), se_explanations['lime'].flatten())
    cbam_gradcam1_lime_score, _ = kendalltau(se_explanations['gradcam1'].flatten(), se_explanations['lime'].flatten())
    cbam_gradcam2_lime_score, _ = kendalltau(se_explanations['gradcam2'].flatten(), se_explanations['lime'].flatten())
    cbam_gradcam3_lime_score, _ = kendalltau(se_explanations['gradcam3'].flatten(), se_explanations['lime'].flatten())
    cbam_gradcam4_lime_score, _ = kendalltau(se_explanations['gradcam4'].flatten(), se_explanations['lime'].flatten())
    cbam_gradcam0_ig_score, _ = kendalltau(se_explanations['gradcam0'].flatten(), se_explanations['ig'].flatten())
    cbam_gradcam1_ig_score, _ = kendalltau(se_explanations['gradcam1'].flatten(), se_explanations['ig'].flatten())
    cbam_gradcam2_ig_score, _ = kendalltau(se_explanations['gradcam2'].flatten(), se_explanations['ig'].flatten())
    cbam_gradcam3_ig_score, _ = kendalltau(se_explanations['gradcam3'].flatten(), se_explanations['ig'].flatten())
    cbam_gradcam4_ig_score, _ = kendalltau(se_explanations['gradcam4'].flatten(), se_explanations['ig'].flatten())
    cbam_lime_ig_score, _ = kendalltau(cbam_explanations['lime'].flatten(), cbam_explanations['ig'].flatten())

    model_names1.append('CBAM')
    gradcam0_lime_scores.append(cbam_gradcam0_lime_score)
    gradcam1_lime_scores.append(cbam_gradcam1_lime_score)
    gradcam2_lime_scores.append(cbam_gradcam2_lime_score)
    gradcam3_lime_scores.append(cbam_gradcam3_lime_score)
    gradcam4_lime_scores.append(cbam_gradcam4_lime_score)
    gradcam0_ig_scores.append(cbam_gradcam0_ig_score)
    gradcam1_ig_scores.append(cbam_gradcam1_ig_score)
    gradcam2_ig_scores.append(cbam_gradcam2_ig_score)
    gradcam3_ig_scores.append(cbam_gradcam3_ig_score)
    gradcam4_ig_scores.append(cbam_gradcam4_ig_score)
    lime_ig_scores.append(cbam_lime_ig_score)
    
    # Calculating the similarities between explanation techniques and attention weights
    for layer_num in range(0, 5):
        for explanation_name in explanation_names:
            if explanation_name == 'gradcam':
                explanation_name = explanation_name + str(layer_num)
            
            # SE
            se_explanation = se_explanations[explanation_name]
            
            # Channel attention
            se_attention_tensor = se_model.get_layer('SE_block' + layer_num).attention_map
            se_attention_resized = se_attention_tensor[0, 0, 0, :]
            se_attention_array = se_attention_resized.numpy()
            se_attention_condensed = condense(se_attention_array)
            se_attention_weights = scale_up(se_attention_condensed)
            
            se_channel_score, _ = kendalltau(se_explanation.flatten(), se_attention_weights.flatten())

            # CBAM
            cbam_explanation = cbam_explanations[explanation_name]
            
            # Channel attention
            cbam_channel_attention_tensor = cbam_model.get_layer('CBAM_block' + layer_num).channel.channel_attention_map
            cbam_channel_attention_resized = cbam_channel_attention_tensor[0, 0, 0, :]
            cbam_channel_attention_array = cbam_channel_attention_resized.numpy()
            cbam_channel_attention_condensed = condense(cbam_channel_attention_array)
            cbam_channel_attention_weights = scale_up(cbam_channel_attention_condensed)

            cbam_channel_score, _ = kendalltau(cbam_explanation.flatten(), cbam_channel_attention_weights.flatten())

            # Convolutional Block Attention Module spatial attention
            cbam_spatial_attention_tensor = cbam_model.get_layer('CBAM_block' + layer_num).spatial.spatial_attention_map
            cbam_spatial_attention_array = cbam_spatial_attention_tensor.numpy()
            cbam_spatial_attention_weights = scale_up(cbam_spatial_attention_array)

            cbam_spatial_score, _ = kendalltau(cbam_explanation.flatten(), cbam_spatial_attention_weights.flatten())
    
            # Convolutional Block Attention Module channel-spatial attention
            cbam_spatial_channel_attention_array = cbam_channel_attention_condensed * cbam_spatial_attention_array
            cbam_spatial_channel_attention_weights = scale_up(cbam_spatial_channel_attention_array)

            cbam_spatial_channel_score, _ = kendalltau(cbam_explanation.flatten(), cbam_spatial_channel_attention_weights.flatten())
            
            # Storing results
            attention_types.append('channel')
            explanation_types.append(explanation_name)
            layer_nums.append(layer_num)
            model_names.append('SE')
            scores.append(se_channel_score)

            attention_types.append('channel')
            explanation_types.append(explanation_name)
            layer_nums.append(layer_num)
            model_names.append('CBAM')
            scores.append(cbam_channel_score)

            attention_types.append('spatial')
            explanation_types.append(explanation_name)
            layer_nums.append(layer_num)
            model_names.append('CBAM')
            scores.append(cbam_spatial_score)

            attention_types.append('channel_spatial')
            explanation_types.append(explanation_name)
            layer_nums.append(layer_num)
            model_names.append('CBAM')
            scores.append(cbam_spatial_channel_score)

    image_processed += 1

    # Saving data to file if enough images was processed
    if image_processed >= image_treshold:
        attention_dataframe = {'model': model_names, 'attention_type': attention_types, 'layer': layer_nums, 'explanation': explanation_types, 'score': scores}
        explanation_dataframe = {'model': model_names1, 'gradcam_lime': gradcam_lime_scores, 'gradcam_ig': gradcam_ig_scores, 'lime_ig': lime_ig_scores}

        attention_dataframe.to_hdf("similarity_scores.h5", key="attention", mode="a")
        explanation_dataframe.to_hdf("similarity_scores.h5", key="explanation", mode="a")

        model_names.clear()
        attention_types.clear()
        layer_nums.clear()
        explanation_types.clear()
        scores.clear()
        
        model_names1.clear()
        gradcam_lime_scores.clear()
        gradcam_ig_scores.clear()
        lime_ig_scores.clear()

        model_names1.clear()
        gradcam0_lime_scores.clear()
        gradcam1_lime_scores.clear()
        gradcam2_lime_scores.clear()
        gradcam3_lime_scores.clear()
        gradcam4_lime_scores.clear()
        gradcam0_ig_scores.clear()
        gradcam1_ig_scores.clear()
        gradcam2_ig_scores.clear()
        gradcam3_ig_scores.clear()
        gradcam4_ig_scores.clear()
        lime_ig_scores.clear()
        
        image_processed = 0

NameError: name 'functional_cbam_model' is not defined